In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
from PIL import Image,ImageTk

Unsupervised Fabric Defect Detection Based on Self-Feature Comparison

In [2]:
def defect_detect(image_path):
    #reading and copying the input image
    input_image = cv2.imread(image_path)
    img = input_image.copy()
    
    #convert the image to HSV color space
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    h = hsv[:,:,0]
    s = hsv[:,:,1]
    v = hsv[:,:,2]

    #apply blurring and denoising
    blr = cv2.blur(v,(15,15))
    dst = cv2.fastNlMeansDenoising(blr,None,10,7,21)

    #thresholding
    _, binary = cv2.threshold(dst,127,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    #define a kernel for erosion an dilation
    kernel = np.ones((5,5),np.uint8)

    #erosion and dilation
    erosion = cv2.erode(binary,kernel,iterations = 1)
    dilation = cv2.dilate(binary,kernel,iterations=1)

    if (dilation == 0).sum() > 1:
        print('Defective fabric')
        contours, _ = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for i in contours:
            if cv2.contourArea(i)<261121.0:
                cv2.drawContours(img,i,-1,(0,255,0),3)
    else:
        print("Good fabric")

    return input_image, img




In [3]:
def browse_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.gif")])
    if file_path:
        input_image, img = defect_detect(file_path)

        # Display the results
        result_window = tk.Toplevel(root)
        result_window.title("Defect Detection Results")

        # Show the input image
        input_image_label = tk.Label(result_window, text="Input Image:")
        input_image_label.pack()
        input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
        input_image_photo = ImageTk.PhotoImage(Image.fromarray(input_image))
        input_image_label = tk.Label(result_window, image=input_image_photo)
        input_image_label.image = input_image_photo
        input_image_label.pack()

        # Show defect locations
        defect_label = tk.Label(result_window, text="Defective Regions:")
        defect_label.pack()
        region_image = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        region_image_photo = ImageTk.PhotoImage(Image.fromarray(region_image))
        region_label = tk.Label(result_window, image=region_image_photo)
        region_label.image = region_image_photo
        region_label.pack()

        result_window.mainloop()

In [4]:
# Create the main UI window
root = tk.Tk()
root.title("Fabric Defect Detection")

# Create a button to browse for an image
browse_button = tk.Button(root, text="Browse Image", command=browse_image)
browse_button.pack()

root.mainloop()

Defective fabric
